# Add calculated ionization rates to KROME's datafile 

In [1]:
from Photon_Ionization import *

In [7]:
distance = np.array([1,2,4,8])
dic = {'alpha':0.1, 'beta':100, 'mdot':3}
Dat = compare(distance, dic)

## Column density vs. Ionization rate

In [2]:
dataH = np.loadtxt('ratexH.dat')
dataHe = np.loadtxt('ratexHe.dat')
dataH[:,1] += dataH[:,0] 
dataH = 10**dataH
dataHe[:,1] += dataHe[:,0] 
dataHe = 10**dataHe

### Column density vs. Number density
$$N^i=n^i\sqrt{\frac{\pi kT}{G\rho\mu m_H}}$$
Thus
$$
\frac{N_H}{N_{He}}=\frac{n_H}{n_{He}}\equiv\chi
$$
Furthurmore
$$
\rho\mu m_H=\frac{(m_Hn_H+m_{He}n_{He})^2}{n_H+n_{He}}=n_H\frac{(m_H\chi+m_{He})^2}{\chi+1}
$$

In [3]:
def num2col(nH, nHe):
    chi = nH/nHe
    A = np.sqrt(np.pi*k*T/G)
    B = np.sqrt((mH*chi+mHe)**2/(chi+1)*nH)
    return(nH*A/B, nHe*A/B)
def col2num(NH, NHe):
    chi = NH/NHe
    C = np.sqrt(np.pi*k*T/G/(mH*chi+mHe)**2*(chi+1))
    nH = (NH/C)**2
    nHe = nH/chi
    return(nH, nHe)

In [4]:
dataH[:,0], dataH[:,1] = col2num(dataH[:,0], dataH[:,1])
dataHe[:,0], dataHe[:,1] = col2num(dataHe[:,0], dataHe[:,1])
daH, daHe = dataH.copy(), dataHe.copy()

In [5]:
ion = np.array([])
for nH, nHe in daH[:,0:2]:
    ion = np.append(ion, Ion_p(H, He, H, None, nH, nHe))
for i in range(len(ion)):
    print(np.log10(ion[i]), np.log10(daH[:,-1][i]))

-18.8189764698801 -18.818976
-18.818976462451943 -18.818976
-18.81897645648546 -18.818976
-18.81897645174478 -18.818976
-18.818976448042505 -18.818976
-18.818976445232277 -18.818976
-18.818976443203006 -18.818976
-18.818976441874465 -18.818976
-18.818976441194142 -18.818976
-18.818976441135142 -18.818976
-18.81897644169513 -18.818976
-18.818976442896247 -18.818976
-18.81897644478597 -18.818976
-18.81897644743901 -18.818976
-18.818976450960236 -18.818976
-18.81897645548885 -18.818976
-18.818976461203874 -18.818976
-18.818976468331236 -18.818976
-18.81897647715268 -18.818976
-18.818976488016922 -18.818976
-18.81897650135346 -18.818976
-18.818976517689496 -18.818976
-18.818976537670828 -18.818976
-18.818976562087318 -18.818976
-18.818976591904153 -18.818976
-18.818976628300103 -18.818976
-18.818976672713912 -18.818976
-18.818976726901315 -18.818976
-18.818976793004456 -18.818976
-18.818976873636394 -18.818976
-18.818976523706446 -18.818977
-18.818976507273746 -18.818977
-18.81897649407458

In [8]:
ion = np.array([])
Dat[1].intro()
print(Dat[1].distance_kpc)
for nH, nHe in daH[:,0:2]:
    ion = np.append(ion, Ion_p(H, He, H, Dat[1], nH, nHe))
for i in range(len(ion)):
    print(np.log10(daH[:,-1][i]), np.log10(ion[i]))

************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
4.0
-18.818976 -17.8410211331627
-18.818976 -17.841021063811738
-18.818976 -17.841021008146825
-18.818976 -17.841020963967466
-18.818976 -17.841020929527183
-18.818976 -17.841020903464532
-18.818976 -17.841020884749213
-18.818976 -17.84102087264138
-18.818976 -17.841020866662404
-18.818976 -17.84102086657594
-18.818976 -17.841020872378547
-18.818976 -17.841020884299624
-18.818976 -17.84102090281041
-18.818976 -17.8410209286427
-18.818976 -17.841020962817634
-18.818976 -17.841021006686198
-18.818976 -17.841021061982573
-18.818976 -17.841021130892734
-18.818976 -17.84102121614069
-18.818976 -17.841021321096292
-18.818976 -17.841021449908794
-18.818976 -17.841021607670168
-18.818976 -17.841021800616993
-18.818976 -17.84102203637641
-18.818976 -17.841022324267897
-18.818976 -17.841022675672672
-18.818976 -17.841023104481657
-18.818976 -17.841023627645832
-18.818976 -17.84102426584638
-18.818976 -17.84102